In [1]:
import findspark 
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("local").setAppName("DFJoin")

from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

22/05/09 22:09:33 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/09 22:09:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/09 22:09:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 22:09:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/09 22:09:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/09 22:09:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/09 22:09:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
products = [ 
          # (product_id, product_name, brand_id)  
         (1, 'iPhone', 100),
         (2, 'Galaxy', 200),
         (3, 'Redme', 300), #   no matching brand
         (4, 'Pixel', 400),
]

brands = [
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"), # no matching products
]
 
productDf = spark.createDataFrame(data=products, schema=["product_id", "product_name", "brand_id"])
brandDf = spark.createDataFrame(data=brands, schema=["brand_id", "brand_name"])
productDf.show()
brandDf.show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+



In [5]:
# Inner Join
# productDf is left
# brandDf is right
# inner join: select/pick only matching record, discord if no matches found
# join returns a new df
df = productDf.join(brandDf, productDf.brand_id  ==  brandDf["brand_id"], "inner") 
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand_id: long (nullable = true)
 |-- brand_id: long (nullable = true)
 |-- brand_name: string (nullable = true)



+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
+----------+------------+--------+--------+----------+



In [7]:
# how to remove specific column
df = productDf.join(brandDf, productDf.brand_id  ==  brandDf["brand_id"], "inner")\
              .drop(brandDf["brand_id"])      
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand_id: long (nullable = true)
 |-- brand_name: string (nullable = true)



+----------+------------+--------+----------+
|product_id|product_name|brand_id|brand_name|
+----------+------------+--------+----------+
|         1|      iPhone|     100|     Apple|
|         2|      Galaxy|     200|   Samsung|
|         4|       Pixel|     400|    Google|
+----------+------------+--------+----------+



In [8]:
# Outer Join, Full Outer Outer, [Left outer + Right outer]
# pick all records from left dataframe, and also right dataframe
# if no matches found, it fills null data for not matched records
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "outer").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+



In [9]:
# Left, Left Outer join 
# picks all records from left, if no matches found, it fills null for right data
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftouter").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+



In [10]:
# Right, Right outer Join
# picks all the records from right, if no matches found, fills left data with null
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "rightouter").show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
+----------+------------+--------+--------+----------+



In [11]:
# left semi join
# join in general convention, it pull the records from both right and left, join them based on condition
# left semi join, join left and right based on condition, however it pull the records only from left side

# it is similar to innerjoin, but pick/project records only from left
# we can't see brand_id, brand_name from brands df
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftsemi").show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         4|       Pixel|     400|
+----------+------------+--------+



In [12]:
# left anti join: exact opposite to semi join
# picks the records that doesn't have match on the right side
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftanti").show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         3|       Redme|     300|
+----------+------------+--------+



In [13]:

store = [
    #(store_id, store_name)
    (1000, "Poorvika"),
    (2000, "Sangeetha"),
    (4000, "Amazon"),
    (5000, "FlipKart"), 
]
 
storeDf = spark.createDataFrame(data=store, schema=["store_id", "store_name"])
storeDf.show()



+--------+----------+
|store_id|store_name|
+--------+----------+
|    1000|  Poorvika|
|    2000| Sangeetha|
|    4000|    Amazon|
|    5000|  FlipKart|
+--------+----------+



In [14]:

# cartesian , take row from left side, pair with all from right side
productDf.crossJoin(storeDf).show()

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|store_id|store_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|    1000|  Poorvika|
|         1|      iPhone|     100|    2000| Sangeetha|
|         1|      iPhone|     100|    4000|    Amazon|
|         1|      iPhone|     100|    5000|  FlipKart|
|         2|      Galaxy|     200|    1000|  Poorvika|
|         2|      Galaxy|     200|    2000| Sangeetha|
|         2|      Galaxy|     200|    4000|    Amazon|
|         2|      Galaxy|     200|    5000|  FlipKart|
|         3|       Redme|     300|    1000|  Poorvika|
|         3|       Redme|     300|    2000| Sangeetha|
|         3|       Redme|     300|    4000|    Amazon|
|         3|       Redme|     300|    5000|  FlipKart|
|         4|       Pixel|     400|    1000|  Poorvika|
|         4|       Pixel|     400|    2000| Sangeetha|
|         4|       Pixel|     400|    4000|    Amazon|
|         